In [348]:
import tensorflow as tf
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

<h2>Extract MNIST data</h2>
<p style="font-size:20px">You can change the option of one_hot encoding.

In [349]:
from tensorflow.examples.tutorials.mnist import input_data
#get mnist data, with one_hot encoding
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)
#suppress warnings
tf.logging.set_verbosity(old_v)

num_train = mnist.train.num_examples #55,000
num_validation = mnist.validation.num_examples #5000
num_test = mnist.test.num_examples #10,000

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


<h2>Define hyperparameters</h2>

In [375]:
#learning rate
#lr = 0.005
lr = 0.005
#number of traning steps
#num_steps =1750
#number of batch_size
#batch_size = 32
num_steps = 2000
batch_size = 32

#network parameters
n_hidden_1 = 4000
n_hidden_2 = 4000
#n_hidden_3 = 1200
num_input = 784
num_classes = 10

<h2>Define placeholder and Variables</h2>

In [376]:
tf.reset_default_graph()

#tf graph input
X = tf.placeholder(tf.float32,[None,num_input],name='X')
Y = tf.placeholder(tf.int32,[None,num_classes],name='Y')
rate = tf.placeholder(tf.float32)

#Layers weight & bias
weights = {
    'W1': tf.Variable(tf.random_normal([num_input, n_hidden_1]),name='W1'),
    'W2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]),name='W2'),
    #'W3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3]),name='W3'),
    'Wout': tf.Variable(tf.random_normal([n_hidden_2, num_classes]),name='Wout')
}

biases = {
    'b1': tf.Variable(tf.zeros(shape=[n_hidden_1]),name='b1'),
    'b2': tf.Variable(tf.zeros(shape=[n_hidden_2]),name='b2'),
    #'b3': tf.Variable(tf.zeros(shape=[n_hidden_3]),name='b3'),
    'bout': tf.Variable(tf.zeros(shape=[num_classes]),name='bout')
}

<h2>Define neural network</h2>

In [377]:
#define a neural net model
def neural_net(x):
    
    layer_1_out = tf.add(tf.matmul(x,weights['W1']),biases['b1'])
    #drop_out_temp = tf.nn.dropout(layer_1_out)
    layer_1_out = tf.nn.relu(layer_1_out)
    drop_out = tf.layers.dropout(layer_1_out, rate)
    
    #layer_1_out = tf.nn.sigmoid(layer_1_out)
    layer_2_out = tf.add(tf.matmul(drop_out,weights['W2']),biases['b2'])
    layer_2_out = tf.nn.relu(layer_2_out)
    
    #layer_3_out = tf.add(tf.matmul(layer_2_out,weights['W3']),biases['b3'])
    #layer_3_out = tf.nn.relu(layer_3_out)
    
    drop_out = tf.layers.dropout(layer_2_out, rate)
    out = tf.add(tf.matmul(drop_out,weights['Wout']),biases['bout'])
    return out

<h2>Define cost function and accuracy</h2>

In [378]:

#predicted labels
logits = neural_net(X)

#define loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=Y),name='loss')
#define optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
train_op = optimizer.minimize(loss)

#compare the predicted labels with true labels
correct_pred = tf.equal(tf.argmax(logits,1),tf.argmax(Y,1))

#compute the accuracy by taking average
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')

#Initialize the variables
init = tf.global_variables_initializer()

<h2>Execute training</h2>

In [381]:

with tf.Session() as sess:
    sess.run(init)
    
    for i in range(num_steps):
        #fetch batch
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        #run optimization
        sess.run(train_op, feed_dict={X:batch_x, Y:batch_y, rate: 0.5})
        if i % 100 ==0:
            acc = sess.run(accuracy,feed_dict={X:batch_x, Y:batch_y, rate: 0.5})
            print("step "+str(i)+", Accuracy= {:.3f}".format(acc))
            #print(mnist.train.shape())
    
    print("Training finished!")
    
    print("Testing ACcuracy:", sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels, rate: 1}))

step 0, Accuracy= 0.406
step 100, Accuracy= 0.938
step 200, Accuracy= 0.938
step 300, Accuracy= 1.000
step 400, Accuracy= 1.000
step 500, Accuracy= 1.000
step 600, Accuracy= 0.938
step 700, Accuracy= 1.000
step 800, Accuracy= 1.000
step 900, Accuracy= 0.969
step 1000, Accuracy= 1.000
step 1100, Accuracy= 1.000
step 1200, Accuracy= 1.000
step 1300, Accuracy= 1.000
step 1400, Accuracy= 1.000
step 1500, Accuracy= 1.000
step 1600, Accuracy= 1.000
step 1700, Accuracy= 1.000
step 1800, Accuracy= 0.969
step 1900, Accuracy= 1.000
Training finished!
Testing ACcuracy: 0.9601


<h2>Your results</h2>

During this project, I stated from the example of the MNIST illustrated in the classroom. First of all, the example lacks of activation function in the hidden layers so I add two reLu functions as the activation functions of the two hidden layers, which imporved the training accuracy and testing accuracy a little and both of the accuracies are below 90%. I then added another hidden layer to the networks, which to my surprise decreased the accuracy, so I removed the third hidden layers. Then, I decreased the learning rate to 0.005, which imporved the performance of the network. Then, I realized that the depth of a single layer should affect the results, so I increased the hidden layer 1 and 2 to 4000 neurons each, which significantly imporved the performance of the code: the training results are approaching 100% and the testing accuracy is around 96%.

|Trial Numbers|learning rate|number of neurons in one layer|number of layers|drop out ratio|test accuracy|
|---|---|---|---|---|---|
|trial1|0.5|300|2|0|85%|
|trial2|0.01|300|2|0|Increase|
|trial3|0.005|300|2|0|Increase|
|trial4|0.001|300|2|0|No change|
|trial5|0.005|200|2|0|Decrease|
|trial6|0.005|1000|2|0|Increase|
|trial7|0.005|2000|2|0|Increase|
|trial8|0.005|3000|2|0|Increase|
|trial9|0.005|4000|2|0|No change|
|trial10|0.005|3000|3|0|Decrease|
|trial11|0.005|3000|2|0.7|Increase|
|trial12|0.005|3000|3|0.5|Increase|
|trial13|0.005|3000|3|0.3|Decrease|
|Optimal|0.005|3000|3|0.5|96%|